# Gas Properties Calculator Notebook

This notebook demonstrates the calculation of gas specific gravity and compressibility factor (Z-Factor) using the Standing & Katz correlation. The notebook is organized into several sections:

- **Introduction:** Overview of the notebook.
- **Library Imports:** Importing necessary libraries.
- **Function Definitions:** Functions to compute specific gravity, gas properties, and Z-Factor.
- **Visualization:** Plotting the Z-Factor chart.
- **Saving Reports:** Appending daily reports to an Excel file.

Each section is explained in detail using Markdown, and the code is provided in executable cells.


In [2]:
# Import necessary libraries for our project
import tkinter as tk
from tkinter import messagebox, filedialog
from datetime import datetime
import numpy as np
import scipy.interpolate
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd
import os

# Enable inline plotting (only needed if you plan to display matplotlib plots in the notebook)
%matplotlib inline


## Function Definitions

In the following cells, we define several functions:

1. **`standing_katz_z_factor`:**  
   Computes the Z-Factor using interpolation between tabulated values for various pseudo-reduced temperatures (Tpr) and pressures (Ppr).

2. **`calculate_sg_from_composition`:**  
   Calculates the gas specific gravity (SG) based on the composition fractions of CH₄, C₂H₆, CO₂, and N₂.

3. **`calculate_gas_properties`:**  
   Uses the specific gravity along with the provided pressure and temperature to compute pseudo-reduced properties and then the Z-Factor.

4. **`plot_z_factor_chart`:**  
   Generates a plot of the Standing & Katz Z-Factor chart and highlights the computed Z-Factor point.

5. **`save_report`:**  
   Saves the computed properties to an Excel file, appending a new row with the current date to consolidate daily reports.

6. **`clear_fields`:**  
   Resets the input fields and clears any plots.


## Define the standing_katz_z_factor Function

In [4]:
def standing_katz_z_factor(Ppr, Tpr):
    # Tabulated Z-factor data for specific pseudo-reduced temperatures.
    Z_data = {
        1.1: [0.98, 0.94, 0.87, 0.75, 0.65, 0.58, 0.50, 0.46],
        1.2: [0.98, 0.94, 0.89, 0.78, 0.69, 0.61, 0.55, 0.50],
        1.3: [0.98, 0.94, 0.90, 0.81, 0.72, 0.65, 0.59, 0.54],
        1.4: [0.98, 0.95, 0.91, 0.83, 0.75, 0.69, 0.63, 0.58],
        1.5: [0.99, 0.95, 0.92, 0.85, 0.78, 0.72, 0.66, 0.62],
    }
    Ppr_vals = np.array([0.2, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
    Tpr_keys = np.array(sorted(Z_data.keys()))
    
    # Clamp Tpr within available data range.
    if Tpr <= Tpr_keys[0]:
        Tpr_low = Tpr_high = Tpr_keys[0]
    elif Tpr >= Tpr_keys[-1]:
        Tpr_low = Tpr_high = Tpr_keys[-1]
    else:
        for i in range(len(Tpr_keys) - 1):
            if Tpr_keys[i] <= Tpr <= Tpr_keys[i+1]:
                Tpr_low = Tpr_keys[i]
                Tpr_high = Tpr_keys[i+1]
                break
    
    Z_low = np.interp(Ppr, Ppr_vals, Z_data[Tpr_low])
    if Tpr_low == Tpr_high:
        return Z_low
    Z_high = np.interp(Ppr, Ppr_vals, Z_data[Tpr_high])
    Z = Z_low + (Z_high - Z_low) * ((Tpr - Tpr_low) / (Tpr_high - Tpr_low))
    return Z


## Define Additional Functions

In [5]:
def calculate_sg_from_composition():
    try:
        ch4 = float(entry_ch4.get())
        c2h6 = float(entry_c2h6.get())
        co2 = float(entry_co2.get())
        n2 = float(entry_n2.get())
        
        total_fraction = ch4 + c2h6 + co2 + n2
        if abs(total_fraction - 1.0) > 0.01:
            messagebox.showerror("Input Error", "Gas composition fractions must sum to 1.")
            return None
        
        mw_ch4, mw_c2h6, mw_co2, mw_n2 = 16.04, 30.07, 44.01, 28.01
        MW_gas = (ch4 * mw_ch4) + (c2h6 * mw_c2h6) + (co2 * mw_co2) + (n2 * mw_n2)
        SG_gas = MW_gas / 28.96
        entry_sg.delete(0, tk.END)
        entry_sg.insert(0, f"{SG_gas:.4f}")
        return SG_gas
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numerical values for gas composition.")
        return None

def calculate_gas_properties():
    try:
        sg = float(entry_sg.get())
        pressure = float(entry_pressure.get())
        temperature = float(entry_temperature.get())
        
        Ppc = 677 + (15 * sg)
        Tpc = 168 + (325 * sg)
        
        Ppr = pressure / Ppc
        Tpr = temperature / Tpc
        
        Z = standing_katz_z_factor(Ppr, Tpr)
        
        result_text.set(f"Pseudo-reduced Pressure: {Ppr:.2f}\n"
                        f"Pseudo-reduced Temperature: {Tpr:.2f}\n"
                        f"Z-Factor (Standing & Katz): {Z:.4f}")
        return Ppr, Tpr, Z
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numerical values.")
        return None, None, None

def plot_z_factor_chart():
    global canvas
    Ppr, Tpr, Z = calculate_gas_properties()
    if Ppr is None or Tpr is None or Z is None:
        return
    
    fig, ax = plt.subplots(figsize=(6, 5))
    Ppr_values = np.array([0.2, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
    Z_values = {
        1.1: [0.98, 0.94, 0.87, 0.75, 0.65, 0.58, 0.50, 0.46],
        1.2: [0.98, 0.94, 0.89, 0.78, 0.69, 0.61, 0.55, 0.50],
        1.3: [0.98, 0.94, 0.90, 0.81, 0.72, 0.65, 0.59, 0.54],
        1.4: [0.98, 0.95, 0.91, 0.83, 0.75, 0.69, 0.63, 0.58],
        1.5: [0.99, 0.95, 0.92, 0.85, 0.78, 0.72, 0.66, 0.62],
    }
    for Tpr_key, Z_vals in Z_values.items():
        ax.plot(Ppr_values, Z_vals, marker='o', linestyle='-', label=f'Tpr = {Tpr_key}')
    
    ax.scatter(Ppr, Z, color='red', s=100, label=f'Computed Z (Ppr={Ppr:.2f}, Tpr={Tpr:.2f})')
    ax.set_xlabel("Pseudo-Reduced Pressure (Ppr)")
    ax.set_ylabel("Compressibility Factor (Z)")
    ax.set_title("Standing & Katz Z-Factor Chart")
    ax.legend()
    ax.grid(True)
    
    if canvas:
        canvas.get_tk_widget().destroy()
    canvas = FigureCanvasTkAgg(fig, master=frame_chart)
    canvas.draw()
    canvas.get_tk_widget().pack()

def save_report():
    filename = "Gas_Properties_Daily_Report.xlsx"
    current_date = datetime.now().strftime('%Y-%m-%d')
    try:
        computed_z = result_text.get().split("\n")[-1].split(": ")[-1]
    except Exception:
        computed_z = ""
    
    data = {
        "Date": [current_date],
        "Methane (CH₄) Fraction": [entry_ch4.get()],
        "Ethane (C₂H₆) Fraction": [entry_c2h6.get()],
        "Carbon Dioxide (CO₂) Fraction": [entry_co2.get()],
        "Nitrogen (N₂) Fraction": [entry_n2.get()],
        "Gas Specific Gravity": [entry_sg.get()],
        "Pressure (psia)": [entry_pressure.get()],
        "Temperature (Rankine)": [entry_temperature.get()],
        "Computed Z-Factor": [computed_z]
    }
    df = pd.DataFrame(data)
    if os.path.exists(filename):
        try:
            existing_df = pd.read_excel(filename)
            df = pd.concat([existing_df, df], ignore_index=True)
        except Exception as e:
            messagebox.showerror("Error", f"Error reading existing Excel file: {str(e)}")
            return
    try:
        df.to_excel(filename, index=False)
        messagebox.showinfo("Save Successful", "Data saved to Excel successfully!")
    except Exception as e:
        messagebox.showerror("Error", f"Error saving to Excel file: {str(e)}")

def clear_fields():
    entry_ch4.delete(0, tk.END)
    entry_c2h6.delete(0, tk.END)
    entry_co2.delete(0, tk.END)
    entry_n2.delete(0, tk.END)
    entry_sg.delete(0, tk.END)
    entry_pressure.delete(0, tk.END)
    entry_temperature.delete(0, tk.END)
    result_text.set("")
    if canvas:
        canvas.get_tk_widget().destroy()


## GUI Setup Explanation

## Graphical User Interface (GUI) Setup

Below, we create a simple Tkinter-based GUI to interact with the gas properties calculator. This section includes input fields for gas composition, specific gravity, pressure, and temperature, as well as buttons to perform calculations, visualize results, save reports, and clear fields.


## Create the Tkinter GUI

In [8]:
# Setup of the Tkinter GUI
root = tk.Tk()
root.title("Gas Properties Calculator")
root.geometry("750x850")

tk.Label(root, text="Methane (CH₄) Fraction:").pack()
entry_ch4 = tk.Entry(root)
entry_ch4.pack()

tk.Label(root, text="Ethane (C₂H₆) Fraction:").pack()
entry_c2h6 = tk.Entry(root)
entry_c2h6.pack()

tk.Label(root, text="Carbon Dioxide (CO₂) Fraction:").pack()
entry_co2 = tk.Entry(root)
entry_co2.pack()

tk.Label(root, text="Nitrogen (N₂) Fraction:").pack()
entry_n2 = tk.Entry(root)
entry_n2.pack()

tk.Label(root, text="Gas Specific Gravity:").pack()
entry_sg = tk.Entry(root)
entry_sg.pack()

tk.Label(root, text="Pressure (psia):").pack()
entry_pressure = tk.Entry(root)
entry_pressure.pack()

tk.Label(root, text="Temperature (Rankine):").pack()
entry_temperature = tk.Entry(root)
entry_temperature.pack()

btn_calculate_sg = tk.Button(root, text="Calculate SG", command=calculate_sg_from_composition)
btn_calculate_sg.pack()

btn_calculate = tk.Button(root, text="Calculate Gas Properties", command=calculate_gas_properties)
btn_calculate.pack()

btn_visualize = tk.Button(root, text="Visualize Z-Factor Chart", command=plot_z_factor_chart)
btn_visualize.pack()

btn_save = tk.Button(root, text="Save Properties", command=save_report)
btn_save.pack()

btn_clear = tk.Button(root, text="Clear Fields", command=clear_fields)
btn_clear.pack()

result_text = tk.StringVar()
tk.Label(root, textvariable=result_text, justify="left").pack()

frame_chart = tk.Frame(root)
frame_chart.pack()

canvas = None

# Uncomment the following line if you want the GUI to run outside of the notebook context.
root.mainloop()
